In [1]:
import pandas as pd
import numpy as np
import math


def distPODs(l,id_source, id_target):
    if id_source == id_target:
        return float(0)
    d = distancias[l][(distancias[l]['source_id_filial_pdr'] == str(id_source)) & (distancias[l]['target_id_filial_pdr'] == str(id_target))]
    if math.isnan(float(d['agg_cost']*0.001)):
        return float(0)
    return float(d['agg_cost']*0.001)


# 1 - Dias da semana que serao processados
S = 7
# 2 - Ordem das visitas (bloco de horas) dentro de cada pacote de trabalho (O)
# e qtd de possíveis pacotes de trabalho (P)
O = 5 
P = 12 
#3 - Categorias e numero de lideres
A = 3
L = 3
# 4 - Separacao dos blocos de horas
H = [0] # bloco de horas que conterão apenas hoteis
PODs = [1,2,3,4] #bloco de horas que contenrão apenas PODs

# pods, auditores, pos_pdr, distancias

L = 1
pods, auditores = [0]*L, [0]*L


pods[0] = pd.read_excel("case_campo_mourao.xlsx", sheet_name = 0)
auditores[0] = pd.read_excel("case_campo_mourao.xlsx", sheet_name = 1)

# pods[1] = pd.read_excel("case_pato_branco.xlsx", sheet_name = 0)
# auditores[1] = pd.read_excel("case_pato_branco.xlsx", sheet_name = 1)

# pods[2] = pd.read_excel("case_toledo.xlsx", sheet_name = 0)
# auditores[2] = pd.read_excel("case_toledo.xlsx", sheet_name = 1)

pos_pdr_table = [0]*L
pos_pdr_table[0] = pd.read_excel("case_campo_mourao.xlsx", sheet_name = 2)
# pos_pdr_table[1] = pd.read_excel("case_pato_branco.xlsx", sheet_name = 2)
# pos_pdr_table[2] = pd.read_excel("case_toledo.xlsx", sheet_name = 2)

m = [0]*L
for l in range(L):
    m[l] = len(pods[l])
    
n = [0]*L
for l in range(L):
    n[l] = len(auditores[l])


# B_box
box_aux = pd.read_csv('historico_box_v2.csv')
b_box = []
for l in range(L):
    b_box_l = []
    for i in pods[l].index:
        for j in box_aux.index:
            if pods[l]["id_filial"][i] == box_aux["filial_id_v2"][j]:
                if box_aux["box_2021"][j] == "#ND":
                    b_box_l.append(int(0))
                b_box_l.append(int(box_aux["box_2021"][j]))
    b_box += [b_box_l]

# Box movel
#box movel    
movel = [0]*L
movel[0] = pd.read_csv('b_box_movel.csv')
# movel[1] = pd.read_csv('b_box_movel_pato_branco.csv')
# movel[2] = pd.read_csv('b_box_movel_toledo.csv')

box_movel = []
for l in range(L):
    box_movel_l = []
    for i in pods[l].index:
        for j in movel[l].index:
            if pods[l]["id_filial"][i] == movel[l]["pod_id"][j]:
                box_movel_l.append(int(movel[l]["box_movel"][j]))
    box_movel += [box_movel_l]


# carregar a distância entre os PODs
distancias = [0]*L

distancias[0] = pd.read_excel("case_campo_mourao.xlsx", sheet_name = 3)
# distancias[1] = pd.read_excel("case_pato_branco.xlsx", sheet_name = 3)
# distancias[2] = pd.read_excel("case_toledo.xlsx", sheet_name = 3)

# criar uma matriz de distancia para cada lider(mxm)
dist = [0]*L
for l in range(L):
    dist[l] = np.zeros((m[l], m[l]))

#carregar as marizes de distancia para cada lider:
for l in range(0,L):
    for i in pods[l].index:
        for j in pods[l].index:
            if pods[l]["id_filial"][i] != pods[l]["id_filial"][j]:
                dseries = distancias[l][(distancias[l]['source_id_filial_pdr'] == pods[l]["id_filial"][i]) & (distancias[l]['target_id_filial_pdr'] == pods[l]["id_filial"][j])]
                d = dseries['agg_cost'].values[0]
                if math.isnan(d):
                    dist[l][i][j] = 0.0001
                else:
                    dist[l][i][j] = d*0.001
            elif pods[l]["id_filial"][i] == pods[l]["id_filial"][j]:
                dist[l][i][j] = 0.0001            

                
                
                
                
################# CRIAR TABELA OUTPUT ########################                
                
#Pacotes de horas
ordem_table = pd.read_excel('./pacoteOrdens22h.xlsx', engine='openpyxl')            
# ordem horas agenda
# indice_semana = {0:'Domingo',1:'Segunda',2:'Terça', 3:'Quarta', 4:'Quinta', 5: 'Sexta', 6:'Sábado'}
indice_semana = {0:'Segunda',1:'Terça',2:'Quarta', 3:'Quinta', 4:'Sexta', 5: 'Sábado', 6:'Domingo'}
########################### selecionando o lider #############################



for l in range(0,1):

#     df_saida = [0]*S
#     #A
#     # Criar data frame com informacoes para saida (saidaFinal) 
#     for s in range(0,S):

#     #     df_saida[s] = pd.read_excel('saidaFinal.xlsx', engine='openpyxl')
#         df_saida[s] = pd.read_excel('saidaFinal_programacao.xlsx', engine='openpyxl') 
        
        
#         #Arquivo txt gerado pela saida
# #         arquivo = open('./saida_semana'+str(s+1)+'_'+str(l)+'.txt','r')
#         arquivo = open('./results-campo_mourao-john_doe_'+str(s+1)+'.txt','r')
#         print('saida_semana'+str(s+1)+'.txt')
#         count = 2
#         for i in arquivo:
#             linha = i.split()

#             if linha[0][0] == 'x':
#                 ls = linha[0].split(',')
#                 ordem = int(ls[1])
#                 if ordem not in H:
#                     pacote = int(ls[4])
#                     ordem = int(ls[1])
#                     pod = int(ls[2])-1
#                     aud = int(ls[3])-1

#                     modulo_hora = ordem_table[(ordem_table['Pacote'] == pacote) & (ordem_table['Ordem'] == ordem)]['modulo'].values[0]#[1]
#                     modulo_entrada = ordem_table[(ordem_table['Pacote'] == pacote) & (ordem_table['Ordem'] == ordem)]['entrada'].values[0]#[1]
#                     modulo_saida = ordem_table[(ordem_table['Pacote'] == pacote) & (ordem_table['Ordem'] == ordem)]['saida'].values[0]#[1]
                    
#                     pod_id = pods[l]['id_filial'][pod] 

#                     pod_LAT = pos_pdr_table[l]['lat'][pod]
#                     pod_LONG = pos_pdr_table[l]['lon'][pod]
#                     cidade = pos_pdr_table[l]['cidade'][pod]

#     #                 pod_LAT = pos_pdr_table[pos_pdr_table['id_filial'] == pod_id]['lat'].values[0]
#     #                 pod_LONG = pos_pdr_table[pos_pdr_table['id_filial'] == pod_id]['lon'].values[0]
#     #                 cidade = pos_pdr_table[pos_pdr_table['id_filial'] == pod_id]['cidade'].values[0]

#                     auditor = auditores[l]['id_consultor'][aud]      

#         #             df_saida[s].loc[count,'Data'] = '0'+str(s+3)+'/01/22'#semana[s]
#                     df_saida[s].loc[count,'Data'] = s+1#semana[s]
#                     df_saida[s].loc[count,'Dia da semana'] = indice_semana[s]
#                     df_saida[s].loc[count,'PDR'] = pod_id
#                     df_saida[s].loc[count,'Bbox'] = b_box[l][pod]
#                     df_saida[s].loc[count,'Box Móvel'] = box_movel[l][pod]
#                     df_saida[s].loc[count,'Cidade'] = cidade
#                     df_saida[s].loc[count,'Consultor'] = auditor
#                     df_saida[s].loc[count,'Latitude'] = pod_LAT
#                     df_saida[s].loc[count,'Longitude'] = pod_LONG
#                     df_saida[s].loc[count,'Módulo'] = str(modulo_hora)+'h'
#                     df_saida[s].loc[count,'Entrada'] = str(modulo_entrada)+':00'
#                     df_saida[s].loc[count,'Saída'] = str(modulo_saida)+':00'
#                     count+=1


#         for ls in range(3,df_saida[s].PDR.count()+2):
#             if df_saida[s]['Consultor'][ls-1] == df_saida[s]['Consultor'][ls]:
#                 pod_i = df_saida[s]['PDR'][int(ls-1)]
#                 pod_j = df_saida[s]['PDR'][int(ls)]
#                 if pod_i != pod_j:
#                     df_saida[s].loc[ls,'Distância'] = distPODs(l,pod_i, pod_j)

#         dist = 0
#         for ls in range(3,df_saida[s].PDR.count()+2):
#             if df_saida[s]['Consultor'][ls-1] == df_saida[s]['Consultor'][ls]:
#                 pod_i = df_saida[s]['PDR'][int(ls-1)]
#                 pod_j = df_saida[s]['PDR'][int(ls)]
#                 if pod_i != pod_j:
#                     dist += distPODs(l,pod_i, pod_j)
#                     df_saida[s].loc[ls,'Distância Acumulada'] = dist
#                     if  ls+1 > df_saida[s].PDR.count():
#                         df_saida[s].loc[ls,'Distância Acumulada'] = dist

#             else:
#                 df_saida[s].loc[ls-1,'Distância Acumulada'] = dist
#                 dist = 0

#     #     df_saida[s].to_excel('./outputs/output_camposmourao_'+str(s+1)+'.xlsx')
#     df_final = [pd.DataFrame(df_saida[s]) for s in range(7)]
#     df_final = pd.concat(df_final)
#     if l == 0:   
#         df_final.to_excel('./outputs/output_camposmourao.xlsx')
#     if l == 1:   
#         df_final.to_excel('./outputs/output_patobranco.xlsx')
#     if l == 2:   
#         df_final.to_excel('./outputs/output_toledo.xlsx')

    ################# POS PROCESSAMENTO ########################

    # removemos um dia de trabalho para cada auditor que trabalha 7 dias
    
#     # Primeira aba do template
#     if l == 0:
#         output = pd.read_excel("./outputs/output_camposmourao.xlsx")
#     if l == 1:
#         output = pd.read_excel("('./outputs/output_patobranco.xlsx")
#     if l == 2:
#         output = pd.read_excel("./outputs/output_toledo.xlsx")


#     id_Consultores = []
#     for i in output.index:
#         if output["Consultor"][i] not in id_Consultores:
#             id_Consultores.append(output["Consultor"][i])


#     Bbox = 10000
#     Box_movel = 100000
#     Data = 0
#     id_consultor_atual = 0
#     for i in id_Consultores:
#         Bbox = 100000
#         Box_movel = 100000
#         Data = 0
#         for k in output.index:
#             if output['Consultor'][k] == i and output['Bbox'][k] <= Bbox and output['Box Móvel'][k] <= Box_movel:
#                 Data = output['Dia da semana'][k]
#                 Bbox = output['Bbox'][k]
#                 Box_movel = output['Box Móvel'][k]
#                 id_consultor_atual = output['Consultor'][k]
#         indice = output[(output['Dia da semana'] == Data) & (output['Consultor'] == i)].index
#         output.drop(indice, inplace=True)
   
#     #Primeira aba template
#     if l == 0:
#         output.to_excel('./outputs/output_camposmourao_6dias.xlsx')
#     if l == 1:
#         output.to_excel('./outputs/output_patobranco_6dias.xlsx')
#     if l == 2:
#         output.to_excel('./outputs/output_toledo_6dias.xlsx')
    
    
    ################# ESCREVER NAS AGENDAS ########################

    if l == 0:
#         cronograma_dia = pd.read_excel("./outputs/output_camposmourao_6dias.xlsx", sheet_name = 0)
        cronograma_dia = pd.read_excel("./outputs/output_camposmourao.xlsx")
    if l == 1:
        cronograma_dia = pd.read_excel("./outputs/patobranco_6dias.xlsx", sheet_name = 0)
    if l == 2:
        cronograma_dia = pd.read_excel("./outputs/output_toledo_6dias.xlsx", sheet_name = 0)

    agendaPDR_modelo = pd.read_excel("./agenda_pdr_modelo.xlsx", sheet_name = 0)
    agendaConsultor_modelo = pd.read_excel("./agenda_consultor_modelo.xlsx", sheet_name = 0)

    coluna_dia = {0:'Segunda',1:'Terça',2:'Quarta', 3:'Quinta', 4:'Sexta', 5: 'Sábado', 6:'Domingo'}
    linha_hora = {'6:00':0, '7:00':1, '8:00':2, '9:00':3, '10:00':4, '11:00':5, '12:00':6, '13:00':7, '14:00':8, '15:00':9, '16:00':10, '17:00':11, '18:00':12, '19:00':13, '20:00':14, '21:00':15, '22:00':16, '23:00':17, '0:00': 18, '1:00':19,'2:00':20,'3:00':21,'4:00':22,'5:00':23}

    df_pdr = [0]*len(pods[l]) 
    #agenda PDR
    for i in pods[l].index:
        df_pdr[i] = agendaPDR_modelo.rename(columns={'PDR: yyyyyyyyy': 'PDR:'+pods[l]['id_filial'][i]})
        
        Pod = pods[l]['id_filial'][i]
        pod_mod = cronograma_dia[cronograma_dia['PDR'] == Pod]['Módulo']
        cont_2_4 = 0
        cont_8 = 0
        for j in pod_mod.index:
            if pod_mod[j] == '8h':
                cont_8 += 1
            if pod_mod[j] == '2h' or pod_mod[j] == '4h':
                cont_2_4 +=1
#         print(f'Para o pod:{Pod} a quantidade com 2h ou 4h: {cont_2_4}' )
#         print(f'Para o pod:{Pod} quantidade com 8h: {cont_8}' )
        
        
        
        for j in cronograma_dia.index:
            if pods[l]['id_filial'][i] == cronograma_dia['PDR'][j]:
                    for k in range(linha_hora[cronograma_dia['Entrada'][j]], linha_hora[cronograma_dia['Saída'][j]]):
                        df_pdr[i].loc[k, coluna_dia[cronograma_dia['Data'][j]-1]] = cronograma_dia['Consultor'][j]
    #     df_pdr[i].to_excel('./agendas_pdr/agenda_pdr_CM_'+str(i)+'.xlsx')

    # result_pdr = pd.concat([df_pdr[i] for i in range(len(pods[l]))], axis=1).reindex(df_pdr[0].index)
    result_pdr = pd.DataFrame()
    
    for i in range(len(pods[l])):
        count = 6
        for index, row in df_pdr[i].iterrows():
            colunaP = df_pdr[l].columns[0]
            h = '{}:00'.format(count)
            if count < 10:
                h = '0{}:00'.format(count)
            count += 1
            linha  = {colunaP:h, 'Segunda': row['Segunda'], 'Terça': row['Terça'], 'Quarta': row['Quarta'], 'Quinta': row['Quinta'], 'Sexta': row['Sexta'], 'Sábado': row['Sábado'], 'Domingo': row['Domingo']  } 
            if count == 23:
                count = 0
            result_pdr = result_pdr.append(linha, ignore_index = True)
        if i < len(pods[l])-1:
            dic = {'PDR:'+pods[l]['id_filial'][0]:'PDR:'+pods[l]['id_filial'][i+1], 'Segunda': 'Segunda', 'Terça': 'Terça', 'Quarta': 'Quarta', 'Quinta': 'Quinta', 'Sexta': 'Sexta', 'Sábado': 'Sábado', 'Domingo': 'Domingo'}
            result_pdr = result_pdr.append(dic, ignore_index=True)

    if l == 0:
        result_pdr.to_excel('./agendas_pdr/agenda_pdr_CM.xlsx')
    if l == 1:
        result_pdr.to_excel('./agendas_pdr/agenda_pdr_PB.xlsx')
    if l == 2:
         result_pdr.to_excel('./agendas_pdr/agenda_pdr_T.xlsx')


    coluna_dia = {0:'Segunda',1:'Terça',2:'Quarta', 3:'Quinta', 4:'Sexta', 5: 'Sábado', 6:'Domingo'}
    linha_hora = {'6:00':0, '7:00':1, '8:00':2, '9:00':3, '10:00':4, '11:00':5, '12:00':6, '13:00':7, '14:00':8, '15:00':9, '16:00':10, '17:00':11, '18:00':12, '19:00':13, '20:00':14, '21:00':15, '22:00':16, '23:00':17, '0:00': 18, '1:00':19,'2:00':20,'3:00':21,'4:00':22,'5:00':23}

    df_consultor = [0]*len(auditores[l]) 
    
    #contador na parte das visitas
    cont_8_total = 0
    cont_2_4_total = 0
    #agenda consultor
    for i in auditores[l].index:
        df_consultor[i] = agendaConsultor_modelo.rename(columns={'Consultor: Felipe': 'Consultor:'+auditores[l]['id_consultor'][i]})
        
        auditor = auditores[l]['id_consultor'][i]
        auditores_mod = cronograma_dia[cronograma_dia['Consultor'] == auditor]['Módulo']
        cont_2_4 = 0
        cont_8 = 0
        for j in auditores_mod.index:
            if auditores_mod[j] == '8h':
                cont_8 += 1
            if auditores_mod[j] == '2h' or auditores_mod[j] == '4h':
                cont_2_4 +=1
        cont_8_total += cont_8
        cont_2_4_total += cont_2_4
        print(f'Para o auditor:{auditor} a quantidade com 2h ou 4h: {cont_2_4}' )
        print(f'Para o auditor:{auditor} quantidade com 8h: {cont_8}' )
        
        for j in cronograma_dia.index:
            if auditores[l]['id_consultor'][i] == cronograma_dia['Consultor'][j]:
                    for k in range(linha_hora[cronograma_dia['Entrada'][j]], linha_hora[cronograma_dia['Saída'][j]]):
                        df_consultor[i].loc[k, coluna_dia[cronograma_dia['Data'][j]-1]] = cronograma_dia['PDR'][j]
    #     df_consultor[i].to_excel('./agendas_consultores/agenda_consultor_CM'+str(i)+'.xlsx')

    result_consultor = pd.DataFrame()
    # result_consultor = pd.concat([df_consultor[i] for i in range(len(auditores[l]))], axis=1).reindex(df_consultor[0].index)
    for i in range(len(auditores[l])):
        count = 6
        for index, row in df_consultor[i].iterrows():
            colunaC = df_consultor[l].columns[0]
            h = '{}:00'.format(count)
            if count < 10:
                h = '0{}:00'.format(count)
            count += 1
            linha  = {colunaC:h, 'Segunda': row['Segunda'], 'Terça': row['Terça'], 'Quarta': row['Quarta'], 'Quinta': row['Quinta'], 'Sexta': row['Sexta'], 'Sábado': row['Sábado'], 'Domingo': row['Domingo']  } 
            if count == 23:
                count = 0
            result_consultor = result_consultor.append(linha, ignore_index = True)
        if i < len(auditores[l])-1:
            dic = {'Consultor:'+str(auditores[l]['id_consultor'][0]):'Consultor:'+str(auditores[l]['id_consultor'][i+1]), 'Segunda': 'Segunda', 'Terça': 'Terça', 'Quarta': 'Quarta', 'Quinta': 'Quinta', 'Sexta': 'Sexta', 'Sábado': 'Sábado', 'Domingo': 'Domingo'}
            result_consultor = result_consultor.append(dic, ignore_index=True)


    if l == 0:
        result_consultor.to_excel('./agendas_consultores/agenda_consultor_CM.xlsx')
    if l == 1:
        result_consultor.to_excel('./agendas_consultores/agenda_consultor_PB.xlsx')
    if l == 2:
        result_consultor.to_excel('./agendas_consultores/agenda_consultor_T.xlsx')
        
    
    
    ### CONTABILIZAR MODULOS DE HORAS ###
    
#     #PDR
#     for i in pods[l].index:
#         Pod = pods[l]['id_filial'][i]
#         pod_mod = cronograma_dia[cronograma_dia['PDR'] == Pod]['Módulo']
#         cont_2_4 = 0
#         cont_8 = 0
#         for j in pod_mod.index:
#             if pod_mod[j] == '8h':
#                 cont_8 += 1
#             if pod_mod[j] == '2h' or pod_mod[j] == '4h':
#                 cont_2_4 +=1
#         print(f'Para o pod:{Pod} a quantidade com 2h ou 4h: {cont_2_4}' )
#         print(f'Para o pod:{Pod} quantidade com 8h: {cont_8}' )
        
#     #Consultor
#     cont_8_total = 0
#     cont_2_4_total = 0
#     for i in auditores.index:
#         auditor = auditores['id_consultor'][i]
#         auditores_mod = cronograma_dia[cronograma_dia['Consultor'] == auditor]['Módulo']
#         cont_2_4 = 0
#         cont_8 = 0
#         for j in auditores_mod.index:
#             if auditores_mod[j] == '8h':
#                 cont_8 += 1
#             if auditores_mod[j] == '2h' or auditores_mod[j] == '4h':
#                 cont_2_4 +=1
#         cont_8_total += cont_8
#         cont_2_4_total += cont_2_4
#         print(f'Para o auditor:{auditor} a quantidade com 2h ou 4h: {cont_2_4}' )
#         print(f'Para o auditor:{auditor} quantidade com 8h: {cont_8}' )

#     print('\n\n')
#     print('No total temos:')
#     print(f'modulos de 2h e 4h: {cont_2_4_total}')
#     print(f'modulos de 8h: {cont_8_total}') 
    

saida_semana1.txt
saida_semana2.txt
saida_semana3.txt
saida_semana4.txt
saida_semana5.txt
saida_semana6.txt
saida_semana7.txt
Para o auditor:7d447c71-5f1b-4442-b9fc-cde39f551479 a quantidade com 2h ou 4h: 6
Para o auditor:7d447c71-5f1b-4442-b9fc-cde39f551479 quantidade com 8h: 3
Para o auditor:ac1f38b1-471a-45b6-b57f-6b02c0c5499f a quantidade com 2h ou 4h: 6
Para o auditor:ac1f38b1-471a-45b6-b57f-6b02c0c5499f quantidade com 8h: 3
Para o auditor:2be12aa2-96d9-4631-b8a7-1b16d3d80b44 a quantidade com 2h ou 4h: 6
Para o auditor:2be12aa2-96d9-4631-b8a7-1b16d3d80b44 quantidade com 8h: 3
Para o auditor:13f30512-5c1b-4c19-be1d-8a161ffc0c19 a quantidade com 2h ou 4h: 8
Para o auditor:13f30512-5c1b-4c19-be1d-8a161ffc0c19 quantidade com 8h: 2
Para o auditor:51c885a9-1d56-49f8-a075-b33124818d01 a quantidade com 2h ou 4h: 10
Para o auditor:51c885a9-1d56-49f8-a075-b33124818d01 quantidade com 8h: 1
